# SSD - Train Model

In [1]:
# Clear GPU
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import os
import time
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from config import opt
from core.res_model import RES18_SSD, RES101_SSD
from core.vgg_model import VGG_SSD
from core.ssd import SSD
from core.resnet import *
from core.utils import detection_collate
from core.multibox_encoder import MultiBoxEncoder
from core.ssd_loss import MultiBoxLoss
from core.voc_dataset import VOCDetection

In [3]:
# Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
# Função para ajustar a taxa de aprendizado durante o treinamento 
# Define a redução da taxa de aprendizado a cada 10 steps
def adjust_learning_rate(optimizer, gamma, step):
    lr = opt.lr * (gamma ** (step))
    print('Alterando a taxa de aprendizado para:', lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [5]:
print('\nExecutando no Device : ', device)


Executando no Device :  cuda:0


In [6]:
# Se a pasta de saída não existir, será criada
if not os.path.exists(opt.save_folder):
    os.mkdir(opt.save_folder)

In [7]:
# Cria instância do modelo
model = SSD(opt.num_classes, opt.anchor_num)

In [8]:
# Carrega o checkpoint ou começa o treinamento do zero
if opt.resume:
    print('Carregando Checkpoint...')
    model.load_state_dict(torch.load(opt.resume))
else:
    vgg_weights = torch.load(opt.basenet)
    print('Carregando a Rede Pré-Treinada...')
    model.vgg.load_state_dict(vgg_weights)

# Envia o modelo para o device     
model.to(device)

Carregando a Rede Pré-Treinada...


SSD(
  (L2Norm): L2Norm()
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddi

In [9]:
# Treina o modelo
model.train()

# Encoder das caixas delimitadoras
mb = MultiBoxEncoder(opt)

# Datasets de treino
image_sets = [['2007', 'trainval']]

# Prepara os dados
dataset = VOCDetection(opt, image_sets = image_sets, is_train = True)

# Carrega os dados
dataloader = torch.utils.data.DataLoader(dataset, batch_size = opt.batch_size, collate_fn = detection_collate, num_workers = 4)

In [10]:
# Calcula o erro
criterion = MultiBoxLoss(opt.num_classes, opt.neg_radio).to(device)

# Otimizador
optimizer = torch.optim.SGD(model.parameters(), lr = opt.lr, momentum = opt.momentum, weight_decay = opt.weight_decay)

In [11]:
start_time = time.time()

print('Iniciando o Treinamento........')

for e in range(opt.epoch):

    # Define se vamos alterar a taxa de aprendizado neste step
    if e % opt.lr_reduce_epoch == 0:
        adjust_learning_rate(optimizer, opt.gamma, e//opt.lr_reduce_epoch)

    # Inicializa totais
    total_loc_loss = 0
    total_cls_loss = 0
    total_loss = 0

    # Loop
    for i , (img, boxes) in enumerate(dataloader):
        img = img.to(device)
        gt_boxes = []
        gt_labels = []
        for box in boxes:
            labels = box[:, 4]
            box = box[:, :-1]
            match_loc, match_label = mb.encode(box, labels)
            gt_boxes.append(match_loc)
            gt_labels.append(match_label)

        gt_boxes = torch.FloatTensor(gt_boxes).to(device)
        gt_labels = torch.LongTensor(gt_labels).to(device)
        p_loc, p_label = model(img)
        loc_loss, cls_loss = criterion(p_loc, p_label, gt_boxes, gt_labels)
        loss = loc_loss + cls_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loc_loss += loc_loss.item()
        total_cls_loss += cls_loss.item()
        total_loss += loss.item()

        if i % opt.log_fn == 0:
            avg_loc = total_loc_loss / (i+1)
            avg_cls = total_cls_loss / (i+1)
            avg_loss = total_loss / (i+1)
            print('Epoch[{}] | batch_idx[{}] | loc_loss [{:.2f}] | cls_loss [{:.2f}] | total_loss [{:.2f}]'.format(e, i, avg_loc, avg_cls, avg_loss))

    # Salva o modelo periodicamente
    if (e+1) % 30 == 0:
        torch.save(model.state_dict(), os.path.join(opt.save_folder, 'modelo-epoch{:02d}-{:.2f}.pth'.format(e, total_loss)))

# Salva a última versão do modelo
torch.save(model.state_dict(), os.path.join(opt.save_folder, 'modelo-final.pth'.format(total_loss)))

print("--- Treinamento finalizado em %s seconds ---" % (time.time() - start_time))

Iniciando o Treinamento........
Alterando a taxa de aprendizado para: 0.0001
Epoch[0] | batch_idx[0] | loc_loss [2.65] | cls_loss [7.44] | total_loss [10.09]
Epoch[0] | batch_idx[1] | loc_loss [2.86] | cls_loss [7.09] | total_loss [9.96]
Epoch[0] | batch_idx[2] | loc_loss [2.92] | cls_loss [6.83] | total_loss [9.75]
Epoch[0] | batch_idx[3] | loc_loss [2.93] | cls_loss [6.53] | total_loss [9.47]
Epoch[0] | batch_idx[4] | loc_loss [2.92] | cls_loss [6.29] | total_loss [9.21]
Epoch[0] | batch_idx[5] | loc_loss [2.88] | cls_loss [6.04] | total_loss [8.92]
Epoch[0] | batch_idx[6] | loc_loss [2.90] | cls_loss [5.81] | total_loss [8.71]
Epoch[0] | batch_idx[7] | loc_loss [2.87] | cls_loss [5.61] | total_loss [8.48]
Epoch[0] | batch_idx[8] | loc_loss [2.87] | cls_loss [5.41] | total_loss [8.28]
Epoch[0] | batch_idx[9] | loc_loss [2.87] | cls_loss [5.23] | total_loss [8.10]
Epoch[0] | batch_idx[10] | loc_loss [2.87] | cls_loss [5.07] | total_loss [7.94]
Epoch[0] | batch_idx[11] | loc_loss [2.86

Epoch[1] | batch_idx[34] | loc_loss [2.50] | cls_loss [3.03] | total_loss [5.53]
Epoch[1] | batch_idx[35] | loc_loss [2.50] | cls_loss [3.03] | total_loss [5.53]
Epoch[1] | batch_idx[36] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.53]
Epoch[1] | batch_idx[37] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[38] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[39] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[40] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[41] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[42] | loc_loss [2.50] | cls_loss [3.04] | total_loss [5.54]
Epoch[1] | batch_idx[43] | loc_loss [2.51] | cls_loss [3.04] | total_loss [5.55]
Epoch[1] | batch_idx[44] | loc_loss [2.51] | cls_loss [3.04] | total_loss [5.55]
Epoch[1] | batch_idx[45] | loc_loss [2.51] | cls_loss [3.04] | total_loss [5.55]
Epoch[1] | batch_idx[46] | l

Epoch[3] | batch_idx[2] | loc_loss [2.33] | cls_loss [2.91] | total_loss [5.24]
Epoch[3] | batch_idx[3] | loc_loss [2.33] | cls_loss [2.92] | total_loss [5.25]
Epoch[3] | batch_idx[4] | loc_loss [2.36] | cls_loss [2.93] | total_loss [5.29]
Epoch[3] | batch_idx[5] | loc_loss [2.36] | cls_loss [2.91] | total_loss [5.27]
Epoch[3] | batch_idx[6] | loc_loss [2.29] | cls_loss [2.92] | total_loss [5.21]
Epoch[3] | batch_idx[7] | loc_loss [2.28] | cls_loss [2.91] | total_loss [5.19]
Epoch[3] | batch_idx[8] | loc_loss [2.28] | cls_loss [2.90] | total_loss [5.18]
Epoch[3] | batch_idx[9] | loc_loss [2.28] | cls_loss [2.90] | total_loss [5.18]
Epoch[3] | batch_idx[10] | loc_loss [2.31] | cls_loss [2.89] | total_loss [5.20]
Epoch[3] | batch_idx[11] | loc_loss [2.32] | cls_loss [2.90] | total_loss [5.22]
Epoch[3] | batch_idx[12] | loc_loss [2.32] | cls_loss [2.89] | total_loss [5.22]
Epoch[3] | batch_idx[13] | loc_loss [2.33] | cls_loss [2.90] | total_loss [5.23]
Epoch[3] | batch_idx[14] | loc_loss 

Epoch[4] | batch_idx[37] | loc_loss [2.31] | cls_loss [2.88] | total_loss [5.19]
Epoch[4] | batch_idx[38] | loc_loss [2.31] | cls_loss [2.88] | total_loss [5.19]
Epoch[4] | batch_idx[39] | loc_loss [2.31] | cls_loss [2.88] | total_loss [5.19]
Epoch[4] | batch_idx[40] | loc_loss [2.31] | cls_loss [2.88] | total_loss [5.20]
Epoch[4] | batch_idx[41] | loc_loss [2.31] | cls_loss [2.88] | total_loss [5.19]
Epoch[4] | batch_idx[42] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[43] | loc_loss [2.29] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[44] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[45] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[46] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[47] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.18]
Epoch[4] | batch_idx[48] | loc_loss [2.30] | cls_loss [2.88] | total_loss [5.19]
Epoch[4] | batch_idx[49] | l

Epoch[6] | batch_idx[5] | loc_loss [2.27] | cls_loss [2.80] | total_loss [5.07]
Epoch[6] | batch_idx[6] | loc_loss [2.33] | cls_loss [2.82] | total_loss [5.15]
Epoch[6] | batch_idx[7] | loc_loss [2.30] | cls_loss [2.83] | total_loss [5.13]
Epoch[6] | batch_idx[8] | loc_loss [2.29] | cls_loss [2.84] | total_loss [5.13]
Epoch[6] | batch_idx[9] | loc_loss [2.27] | cls_loss [2.84] | total_loss [5.10]
Epoch[6] | batch_idx[10] | loc_loss [2.26] | cls_loss [2.84] | total_loss [5.10]
Epoch[6] | batch_idx[11] | loc_loss [2.23] | cls_loss [2.83] | total_loss [5.06]
Epoch[6] | batch_idx[12] | loc_loss [2.22] | cls_loss [2.83] | total_loss [5.05]
Epoch[6] | batch_idx[13] | loc_loss [2.22] | cls_loss [2.83] | total_loss [5.05]
Epoch[6] | batch_idx[14] | loc_loss [2.22] | cls_loss [2.83] | total_loss [5.05]
Epoch[6] | batch_idx[15] | loc_loss [2.24] | cls_loss [2.82] | total_loss [5.07]
Epoch[6] | batch_idx[16] | loc_loss [2.24] | cls_loss [2.83] | total_loss [5.07]
Epoch[6] | batch_idx[17] | loc_lo

...

Epoch[293] | batch_idx[9] | loc_loss [0.77] | cls_loss [1.87] | total_loss [2.64]
Epoch[293] | batch_idx[10] | loc_loss [0.80] | cls_loss [1.88] | total_loss [2.68]
Epoch[293] | batch_idx[11] | loc_loss [0.77] | cls_loss [1.84] | total_loss [2.61]
Epoch[293] | batch_idx[12] | loc_loss [0.77] | cls_loss [1.84] | total_loss [2.61]
Epoch[293] | batch_idx[13] | loc_loss [0.78] | cls_loss [1.85] | total_loss [2.63]
Epoch[293] | batch_idx[14] | loc_loss [0.80] | cls_loss [1.86] | total_loss [2.66]
Epoch[293] | batch_idx[15] | loc_loss [0.80] | cls_loss [1.86] | total_loss [2.67]
Epoch[293] | batch_idx[16] | loc_loss [0.81] | cls_loss [1.87] | total_loss [2.68]
Epoch[293] | batch_idx[17] | loc_loss [0.82] | cls_loss [1.89] | total_loss [2.71]
Epoch[293] | batch_idx[18] | loc_loss [0.81] | cls_loss [1.88] | total_loss [2.68]
Epoch[293] | batch_idx[19] | loc_loss [0.80] | cls_loss [1.86] | total_loss [2.66]
Epoch[293] | batch_idx[20] | loc_loss [0.79] | cls_loss [1.86] | total_loss [2.66]
Epoch

Epoch[294] | batch_idx[41] | loc_loss [0.89] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[42] | loc_loss [0.89] | cls_loss [1.88] | total_loss [2.77]
Epoch[294] | batch_idx[43] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.77]
Epoch[294] | batch_idx[44] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[45] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[46] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[47] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[48] | loc_loss [0.90] | cls_loss [1.88] | total_loss [2.78]
Epoch[294] | batch_idx[49] | loc_loss [0.89] | cls_loss [1.87] | total_loss [2.76]
Epoch[294] | batch_idx[50] | loc_loss [0.89] | cls_loss [1.88] | total_loss [2.76]
Epoch[294] | batch_idx[51] | loc_loss [0.89] | cls_loss [1.88] | total_loss [2.77]
Epoch[294] | batch_idx[52] | loc_loss [0.89] | cls_loss [1.88] | total_loss [2.77]
Epoc

Epoch[296] | batch_idx[6] | loc_loss [0.81] | cls_loss [1.82] | total_loss [2.64]
Epoch[296] | batch_idx[7] | loc_loss [0.82] | cls_loss [1.83] | total_loss [2.65]
Epoch[296] | batch_idx[8] | loc_loss [0.80] | cls_loss [1.82] | total_loss [2.62]
Epoch[296] | batch_idx[9] | loc_loss [0.80] | cls_loss [1.81] | total_loss [2.60]
Epoch[296] | batch_idx[10] | loc_loss [0.82] | cls_loss [1.81] | total_loss [2.64]
Epoch[296] | batch_idx[11] | loc_loss [0.81] | cls_loss [1.82] | total_loss [2.63]
Epoch[296] | batch_idx[12] | loc_loss [0.79] | cls_loss [1.80] | total_loss [2.59]
Epoch[296] | batch_idx[13] | loc_loss [0.80] | cls_loss [1.81] | total_loss [2.61]
Epoch[296] | batch_idx[14] | loc_loss [0.81] | cls_loss [1.83] | total_loss [2.64]
Epoch[296] | batch_idx[15] | loc_loss [0.86] | cls_loss [1.85] | total_loss [2.70]
Epoch[296] | batch_idx[16] | loc_loss [0.86] | cls_loss [1.86] | total_loss [2.72]
Epoch[296] | batch_idx[17] | loc_loss [0.87] | cls_loss [1.87] | total_loss [2.74]
Epoch[29

Epoch[297] | batch_idx[38] | loc_loss [0.83] | cls_loss [1.90] | total_loss [2.72]
Epoch[297] | batch_idx[39] | loc_loss [0.83] | cls_loss [1.90] | total_loss [2.73]
Epoch[297] | batch_idx[40] | loc_loss [0.83] | cls_loss [1.90] | total_loss [2.72]
Epoch[297] | batch_idx[41] | loc_loss [0.82] | cls_loss [1.89] | total_loss [2.71]
Epoch[297] | batch_idx[42] | loc_loss [0.82] | cls_loss [1.89] | total_loss [2.70]
Epoch[297] | batch_idx[43] | loc_loss [0.82] | cls_loss [1.89] | total_loss [2.71]
Epoch[297] | batch_idx[44] | loc_loss [0.83] | cls_loss [1.89] | total_loss [2.71]
Epoch[297] | batch_idx[45] | loc_loss [0.84] | cls_loss [1.89] | total_loss [2.73]
Epoch[297] | batch_idx[46] | loc_loss [0.83] | cls_loss [1.89] | total_loss [2.72]
Epoch[297] | batch_idx[47] | loc_loss [0.83] | cls_loss [1.89] | total_loss [2.73]
Epoch[297] | batch_idx[48] | loc_loss [0.83] | cls_loss [1.89] | total_loss [2.71]
Epoch[297] | batch_idx[49] | loc_loss [0.82] | cls_loss [1.88] | total_loss [2.71]
Epoc

Epoch[299] | batch_idx[3] | loc_loss [0.94] | cls_loss [1.97] | total_loss [2.91]
Epoch[299] | batch_idx[4] | loc_loss [0.91] | cls_loss [1.93] | total_loss [2.85]
Epoch[299] | batch_idx[5] | loc_loss [0.85] | cls_loss [1.86] | total_loss [2.71]
Epoch[299] | batch_idx[6] | loc_loss [0.86] | cls_loss [1.87] | total_loss [2.73]
Epoch[299] | batch_idx[7] | loc_loss [0.85] | cls_loss [1.88] | total_loss [2.73]
Epoch[299] | batch_idx[8] | loc_loss [0.83] | cls_loss [1.87] | total_loss [2.70]
Epoch[299] | batch_idx[9] | loc_loss [0.86] | cls_loss [1.88] | total_loss [2.74]
Epoch[299] | batch_idx[10] | loc_loss [0.87] | cls_loss [1.89] | total_loss [2.76]
Epoch[299] | batch_idx[11] | loc_loss [0.84] | cls_loss [1.87] | total_loss [2.71]
Epoch[299] | batch_idx[12] | loc_loss [0.82] | cls_loss [1.84] | total_loss [2.65]
Epoch[299] | batch_idx[13] | loc_loss [0.83] | cls_loss [1.86] | total_loss [2.69]
Epoch[299] | batch_idx[14] | loc_loss [0.82] | cls_loss [1.86] | total_loss [2.69]
Epoch[299] 